In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
df = pd.read_csv('clean card transaction.csv')

In [3]:
df = df.iloc[:, 1:]

In [4]:
df['Cardnum'] = df['Cardnum'].astype('str')
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

In [5]:
df.dtypes

Recnum                        int64
Cardnum                      object
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                    object
Transtype                    object
Amount                      float64
Fraud                         int64
dtype: object

In [6]:
df['card_merch'] = df['Cardnum'] + df['Merchnum']
df['card_zip'] = df['Cardnum'] + df['Merch zip']
df['card_state'] = df['Cardnum'] + df['Merch state']
df['card_merch_zip'] = df['Cardnum'] + df['Merchnum'] + df['Merch zip']
df['Merch full'] = df['Merchnum'] + df['Merch description']

In [7]:
df

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,card_merch,card_zip,card_state,card_merch_zip,Merch full
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,51421904395509006296254,514219043938118,5142190439TN,5142190439550900629625438118,5509006296254FEDEX SHP 12/23/09 AB#
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,P,31.42,0,514218397361003026333,514218397301803,5142183973MA,51421839736100302633301803,61003026333SERVICE MERCHANDISE #81
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,51421317214503082993600,514213172120706,5142131721MD,5142131721450308299360020706,4503082993600OFFICE DEPOT #191
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,51421484525509006296254,514214845238118,5142148452TN,5142148452550900629625438118,5509006296254FEDEX SHP 12/28/09 AB#
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,51421904395509006296254,514219043938118,5142190439TN,5142190439550900629625438118,5509006296254FEDEX SHP 12/23/09 AB#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96392,96749,5142276053,2010-12-31,3500000006160,BEST BUY 00001610,KY,41042,P,84.79,0,51422760533500000006160,514227605341042,5142276053KY,5142276053350000000616041042,3500000006160BEST BUY 00001610
96393,96750,5142225701,2010-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248,P,118.75,0,51422257018090710030950,514222570145248,5142225701OH,5142225701809071003095045248,8090710030950MARKUS OFFICE SUPPLIES
96394,96751,5142226486,2010-12-31,4503057341100,"TECH PAC, INC",OH,45150,P,363.56,0,51422264864503057341100,514222648645150,5142226486OH,5142226486450305734110045150,"4503057341100TECH PAC, INC"
96395,96752,5142244619,2010-12-31,8834000695412,BUY.COM,CA,92656,P,2202.03,0,51422446198834000695412,514224461992656,5142244619CA,5142244619883400069541292656,8834000695412BUY.COM


In [14]:
df1 = df.copy()
final = df.copy()
df1['check_Date'] = df1.Date
df1['check_Recnum'] = df1.Recnum

In [15]:
df.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud',
       'card_merch', 'card_zip', 'card_state', 'card_merch_zip', 'Merch full'],
      dtype='object')

In [16]:
attributes = list(df.iloc[:,np.r_[1, 3, 5, 6, 10:15]].columns)
attributes

['Cardnum',
 'Merchnum',
 'Merch state',
 'Merch zip',
 'card_merch',
 'card_zip',
 'card_state',
 'card_merch_zip',
 'Merch full']

In [17]:
for entity in attributes:
    df_l = df1[['Recnum', 'Date', entity, 'Amount']]
    df_r = df1[['check_Recnum', 'check_Date', entity, 'Amount']]
    temp = pd.merge(df_l, df_r, left_on = entity, right_on = entity)

    # day since variables
    day_since_df = temp[temp.Recnum > temp.check_Recnum][['Recnum', 'Date', 'check_Date']]\
            .groupby('Recnum')[['Date', 'check_Date']].last()
    mapper = (day_since_df.Date - day_since_df.check_Date).dt.days
    final[entity + '_day_since'] = final.Recnum.map(mapper)
    final[entity + '_day_since'].fillna(365, inplace = True)
    print(f'\n{entity}_day_since ---> Done')

    # Amount variables and count variables
    for offset_t in [0, 1, 3, 7, 14, 30]:
        count_day_df = temp[(temp.check_Date >= (temp.Date - dt.timedelta(offset_t))) & (temp.Recnum >= temp.check_Recnum)]
    #     col_name = f'{entity}_count_{offset_t}'
        mapper_AVG = count_day_df.groupby('Recnum')['Amount_y'].mean()
        mapper_MAX = count_day_df.groupby('Recnum')['Amount_y'].max()
        mapper_MED = count_day_df.groupby('Recnum')['Amount_y'].median()
        mapper_TOL = count_day_df.groupby('Recnum')['Amount_y'].sum()
        final[f'{entity}_Amount_AVG_{offset_t}'] = final.Recnum.map(mapper_AVG)
        final[f'{entity}_Amount_MAX_{offset_t}'] = final.Recnum.map(mapper_MAX)
        final[f'{entity}_Amount_MED_{offset_t}'] = final.Recnum.map(mapper_MED)
        final[f'{entity}_Amount_TOL_{offset_t}'] = final.Recnum.map(mapper_TOL)
        final[f'{entity}_Amount_Actual/AVG_{offset_t}'] = final['Amount']/final[f'{entity}_Amount_AVG_{offset_t}']
        final[f'{entity}_Amount_Actual/MAX_{offset_t}'] = final['Amount']/final[f'{entity}_Amount_MAX_{offset_t}']
        final[f'{entity}_Amount_Actual/MED_{offset_t}'] = final['Amount']/final[f'{entity}_Amount_MED_{offset_t}']
        final[f'{entity}_Amount_Actual/TOL_{offset_t}'] = final['Amount']/final[f'{entity}_Amount_TOL_{offset_t}']
        
        # count variables
        col_name = f'{entity}_count_{offset_t}'
        mapper2 = count_day_df.groupby('Recnum')[entity].count()
        final[col_name] = final.Recnum.map(mapper2)
        print(f'{entity}_count_{str(offset_t)} ---> Done')


Cardnum_day_since ---> Done
Cardnum_count_0 ---> Done
Cardnum_count_1 ---> Done
Cardnum_count_3 ---> Done
Cardnum_count_7 ---> Done
Cardnum_count_14 ---> Done
Cardnum_count_30 ---> Done

Merchnum_day_since ---> Done
Merchnum_count_0 ---> Done
Merchnum_count_1 ---> Done
Merchnum_count_3 ---> Done
Merchnum_count_7 ---> Done
Merchnum_count_14 ---> Done
Merchnum_count_30 ---> Done

Merch state_day_since ---> Done
Merch state_count_0 ---> Done
Merch state_count_1 ---> Done
Merch state_count_3 ---> Done
Merch state_count_7 ---> Done
Merch state_count_14 ---> Done
Merch state_count_30 ---> Done

Merch zip_day_since ---> Done
Merch zip_count_0 ---> Done
Merch zip_count_1 ---> Done
Merch zip_count_3 ---> Done
Merch zip_count_7 ---> Done
Merch zip_count_14 ---> Done
Merch zip_count_30 ---> Done

card_merch_day_since ---> Done
card_merch_count_0 ---> Done
card_merch_count_1 ---> Done
card_merch_count_3 ---> Done
card_merch_count_7 ---> Done
card_merch_count_14 ---> Done
card_merch_count_30 ---> 

In [27]:
key_attr = ['Cardnum','Merch full','Merch state']
for entity1 in key_attr:
    for entity2 in key_attr:
        if entity1 == entity2:
            pass
        else:
            df_l = df1[['Recnum', 'Date', entity1, entity2]]
            df_r = df1[['check_Recnum', 'check_Date', entity1, entity2]]

            temp = pd.merge(df_l, df_r, left_on = entity1, right_on = entity1)

            temp = temp[(temp.Recnum >= temp.check_Recnum)]
            col_name = f'num_of_{entity2}_for_each_{entity1}'
            mapper = temp.groupby(['Recnum', entity1])[entity2+'_y'].nunique()
            mapper = pd.DataFrame(mapper).reset_index()
            final[col_name] = mapper.loc[(mapper['Recnum'] == final['Recnum']) & 
                                         (mapper[entity1] == final[entity1])][entity2+'_y']
            print(f'{entity1} with {entity2} is ------> Done')

Cardnum with Merch full is ------> Done
Cardnum with Merch state is ------> Done
Merch full with Cardnum is ------> Done
Merch full with Merch state is ------> Done
Merch state with Cardnum is ------> Done
Merch state with Merch full is ------> Done


In [28]:
final

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,Merch full_Amount_Actual/MAX_30,Merch full_Amount_Actual/MED_30,Merch full_Amount_Actual/TOL_30,Merch full_count_30,num_of_Merch full_for_each_Cardnum,num_of_Merch state_for_each_Cardnum,num_of_Cardnum_for_each_Merch full,num_of_Merch state_for_each_Merch full,num_of_Cardnum_for_each_Merch state,num_of_Merch full_for_each_Merch state
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,1.0,1.0,1.000000,1,1,1,1,1,1,1
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,P,31.42,0,...,1.0,1.0,1.000000,1,1,1,1,1,1,1
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,1.0,1.0,1.000000,1,1,1,1,1,1,1
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,1.0,1.0,1.000000,1,1,1,1,1,2,2
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,1.0,1.0,0.500000,2,1,1,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96392,96749,5142276053,2010-12-31,3500000006160,BEST BUY 00001610,KY,41042,P,84.79,0,...,1.0,1.0,1.000000,1,4,2,1,1,203,91
96393,96750,5142225701,2010-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248,P,118.75,0,...,1.0,1.0,0.142857,7,35,13,13,1,405,610
96394,96751,5142226486,2010-12-31,4503057341100,"TECH PAC, INC",OH,45150,P,363.56,0,...,1.0,1.0,0.050000,20,60,24,23,1,405,610
96395,96752,5142244619,2010-12-31,8834000695412,BUY.COM,CA,92656,P,2202.03,0,...,1.0,1.0,0.021739,46,22,11,107,1,969,1163


In [30]:
import time

In [31]:
# Velocity change variables:
start = time.time()
for att in attributes:
    for d in ['0','1']:
        for dd in ['3','7','14','30']:
            # Count
            final[att + '_count_' + d + '_by_' + dd] = final[
                att + '_count_' + d]/(final[att + '_count_' + dd] / float(dd))
            # Amount
            final[att + '_Amount_' + d + '_by_' + dd] = final[
                att + '_Amount_TOL_' + d]/(final[att + '_Amount_TOL_' + dd]/ float(dd))
print(f'Total run time: {time.time() - start:0.2f}s')

Total run time: 54.55s


In [32]:
final

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,Merch full_count_0_by_30,Merch full_Amount_0_by_30,Merch full_count_1_by_3,Merch full_Amount_1_by_3,Merch full_count_1_by_7,Merch full_Amount_1_by_7,Merch full_count_1_by_14,Merch full_Amount_1_by_14,Merch full_count_1_by_30,Merch full_Amount_1_by_30
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,30.000000,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,P,31.42,0,...,30.000000,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,30.000000,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,30.000000,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,30.000000,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96392,96749,5142276053,2010-12-31,3500000006160,BEST BUY 00001610,KY,41042,P,84.79,0,...,30.000000,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000
96393,96750,5142225701,2010-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248,P,118.75,0,...,4.285714,4.285714,3.0,3.0,7.000000,7.000000,2.800000,2.800000,4.285714,4.285714
96394,96751,5142226486,2010-12-31,4503057341100,"TECH PAC, INC",OH,45150,P,363.56,0,...,6.000000,6.000000,3.0,3.0,4.666667,4.666667,8.000000,8.000000,6.000000,6.000000
96395,96752,5142244619,2010-12-31,8834000695412,BUY.COM,CA,92656,P,2202.03,0,...,0.652174,0.652174,3.0,3.0,4.666667,4.666667,4.307692,4.307692,2.608696,2.608696


# Day of week risk table variable

In [33]:
import calendar

In [34]:
df['dow'] = df.Date.apply(lambda x: calendar.day_name[x.weekday()])

In [35]:
train_test = df[df.Date < '2010-11-01']

In [36]:
# do statistical smoothing
c = 4
nmid = 20
y_avg = train_test['Fraud'].mean()
y_dow = train_test.groupby('dow')['Fraud'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/c))
df['dow_risk'] = df.dow.map(y_dow_smooth)

In [37]:
y_dow_smooth

dow
Friday       0.025994
Monday       0.008711
Saturday     0.010095
Sunday       0.009630
Thursday     0.018626
Tuesday      0.007127
Wednesday    0.009788
dtype: float64

In [38]:
final['dow_risk'] = df['dow_risk']

In [39]:
final

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,Merch full_Amount_0_by_30,Merch full_count_1_by_3,Merch full_Amount_1_by_3,Merch full_count_1_by_7,Merch full_Amount_1_by_7,Merch full_count_1_by_14,Merch full_Amount_1_by_14,Merch full_count_1_by_30,Merch full_Amount_1_by_30,dow_risk
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,P,31.42,0,...,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96392,96749,5142276053,2010-12-31,3500000006160,BEST BUY 00001610,KY,41042,P,84.79,0,...,30.000000,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994
96393,96750,5142225701,2010-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248,P,118.75,0,...,4.285714,3.0,3.0,7.000000,7.000000,2.800000,2.800000,4.285714,4.285714,0.025994
96394,96751,5142226486,2010-12-31,4503057341100,"TECH PAC, INC",OH,45150,P,363.56,0,...,6.000000,3.0,3.0,4.666667,4.666667,8.000000,8.000000,6.000000,6.000000,0.025994
96395,96752,5142244619,2010-12-31,8834000695412,BUY.COM,CA,92656,P,2202.03,0,...,0.652174,3.0,3.0,4.666667,4.666667,4.307692,4.307692,2.608696,2.608696,0.025994


In [40]:
variables = pd.DataFrame()

In [41]:
variables = final.iloc[:, 15:]
variables['Fraud'] = final['Fraud']
variables.index = final['Recnum']


In [42]:
variables

,Cardnum_day_since,Cardnum_Amount_AVG_0,Cardnum_Amount_MAX_0,Cardnum_Amount_MED_0,Cardnum_Amount_TOL_0,Cardnum_Amount_Actual/AVG_0,Cardnum_Amount_Actual/MAX_0,Cardnum_Amount_Actual/MED_0,Cardnum_Amount_Actual/TOL_0,Cardnum_count_0,...,Merch full_count_1_by_3,Merch full_Amount_1_by_3,Merch full_count_1_by_7,Merch full_Amount_1_by_7,Merch full_count_1_by_14,Merch full_Amount_1_by_14,Merch full_count_1_by_30,Merch full_Amount_1_by_30,dow_risk,Fraud
Recnum,,,,,,,,,,,,,,,,,,,,,
1,365.0,3.62,3.62,3.62,3.62,1.0,1.0,1.0,1.0,1,...,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994,0
2,365.0,31.42,31.42,31.42,31.42,1.0,1.0,1.0,1.0,1,...,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994,0
3,365.0,178.49,178.49,178.49,178.49,1.0,1.0,1.0,1.0,1,...,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994,0
4,365.0,3.62,3.62,3.62,3.62,1.0,1.0,1.0,1.0,1,...,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994,0
5,0.0,3.62,3.62,3.62,7.24,1.0,1.0,1.0,0.5,2,...,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96749,37.0,84.79,84.79,84.79,84.79,1.0,1.0,1.0,1.0,1,...,3.0,3.0,7.000000,7.000000,14.000000,14.000000,30.000000,30.000000,0.025994,0
96750,0.0,118.75,118.75,118.75,237.50,1.0,1.0,1.0,0.5,2,...,3.0,3.0,7.000000,7.000000,2.800000,2.800000,4.285714,4.285714,0.025994,0
96751,1.0,363.56,363.56,363.56,363.56,1.0,1.0,1.0,1.0,1,...,3.0,3.0,4.666667,4.666667,8.000000,8.000000,6.000000,6.000000,0.025994,0


In [43]:
#variables.to_csv('variables_new.csv')